(kl_notebook)=
# La divergenza di Kullback-Leibler

```{admonition} Obiettivi di apprendimento
Dopo aver completato questo capitolo, acquisirete competenze approfondite e specifiche, che vi permetteranno di:

- Comprendere in modo dettagliato il concetto di Expected Log Predictive Density (ELPD), apprezzandone l'importanza e l'applicabilità nel contesto della valutazione predittiva dei modelli statistici.
- Esplorare e analizzare la relazione tra il concetto di entropia e la ELPD, identificando come l'entropia possa influenzare o riflettere la capacità predittiva di un modello attraverso la densità logaritmica predittiva prevista.
```

In questo capitolo, esamineremo in modo dettagliato la Divergenza di Kullback-Leibler ($\mathbb{KL}$) e l'Expected Log Predictive Density (ELPD). Vedremo come questi concetti possano essere utilizzati per valutare e confrontare modelli statistici diversi all'interno di un contesto bayesiano. 

In [1]:
import numpy as np

In [2]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

## Confronto di Modelli Utilizzando la Divergenza di Kullback-Leibler

### Introduzione alla Distribuzione Predittiva Posteriori

La distribuzione predittiva posteriori, denotata come $ Q(\tilde{y} \mid y) $, rappresenta le previsioni su nuovi dati $ \tilde{y} $ basate su un modello statistico $ Q $ e i dati osservati $ y $. Questa distribuzione combina le previsioni del modello per un dato set di parametri $ \theta $ (cioè $ Q(\tilde{y} \mid \theta) $) con la distribuzione posteriore di questi parametri dati i dati osservati (cioè $ P(\theta \mid y) $).

### Misurazione della Divergenza KL

L'obiettivo di utilizzare la divergenza di Kullback-Leibler ($ \mathbb{KL} $) è di quantificare quanto bene la distribuzione predittiva del modello $ Q $ si avvicina alla distribuzione vera $ P $ che ha generato i dati. Matematicamente, ciò è espresso come $ \mathbb{KL}(P \parallel Q) $. Un valore basso di $ \mathbb{KL} $ indica che $ Q $ è una buona approssimazione di $ P $, mentre un valore alto indica una maggiore discrepanza.

### Confronto Pratico tra Modelli

Data la mancanza di conoscenza diretta di $ P $, la vera distribuzione che ha generato i dati, utilizziamo la divergenza $ \mathbb{KL} $ per confrontare diversi modelli. La divergenza KL tra due distribuzioni $ P $ e $ Q $ si calcola come:

$$
\mathbb{KL}(P \parallel Q) = \mathbb{E}_P[\log P] - \mathbb{E}_P[\log Q],
$$

dove $ \mathbb{E}_P $ indica il valore atteso calcolato sotto la distribuzione $ P $.

Per distribuzioni discrete, questo si esprime come:

$$
\mathbb{KL}(P \parallel Q) = \sum_{i=1}^n p_i (\log p_i - \log q_i),
$$

dove $ p_i $ e $ q_i $ rappresentano le probabilità degli eventi $ i $ per le distribuzioni $ P $ e $ Q $ rispettivamente.

La divergenza KL può essere riformulata in termini di valore atteso come segue:

- **Termine $ \log P $**:

  $$
  \mathbb{E}_P[\log P(X)] = \sum_{i=1}^n p_i \log p_i
  $$

- **Termine $ \log Q $**:

  $$
  \mathbb{E}_P[\log Q(X)] = \sum_{i=1}^n p_i \log q_i
  $$

Quindi, la divergenza KL si riduce a:

$$
\mathbb{KL}(P \parallel Q) = \mathbb{E}_P[\log P(X)] - \mathbb{E}_P[\log Q(X)]
$$

Per variabili continue, la formula diventa:

$$
\mathbb{KL}(P \parallel Q) = \int p(x) (\log p(x) - \log q(x)) \, dx.
$$

### Il Termine $ -\mathbb{E}_P[\log Q(y)] $

Quando confrontiamo modelli diversi, il termine $ \mathbb{E}_P[\log P] $ rimane costante e quindi può essere omesso. Ci concentriamo sul termine:

$$
-\mathbb{E}_P[\log Q(y)],
$$

che misura l'adattabilità del modello ai dati osservati. Questo si calcola come:

$$
-\int p(y) \log Q(y) \, dy,
$$

indicando quale modello $ Q $ rappresenti meglio la distribuzione $ P $ secondo la quantità di informazione che si perderebbe utilizzandolo per descrivere i dati osservati.

### Collegamento con l'Expected Log Predictive Density (ELPD)

L'ELPD si focalizza sulla capacità di un modello di predire nuovi dati, offrendo una misura della sua capacità di generalizzazione. L'ELPD è definito come il valore atteso

 del logaritmo della densità predittiva di un modello, calcolato sotto la vera distribuzione dei dati futuri:

$$
\text{ELPD} = \mathbb{E}_{y \sim p(y)} [\log p(y \mid \theta)]
$$

Mentre $ -\int p(y) \log Q(y) \, dy $ quantifica quanto bene un modello descrive la distribuzione attuale dei dati, l'ELPD stima quanto efficacemente il modello può essere utilizzato per prevedere nuovi dati. Questo rende l'ELPD una misura complementare alla $ \mathbb{KL} $, enfatizzando non solo l'adattabilità ma anche la predittività di un modello.

Utilizzare l'ELPD come criterio di valutazione tende a favorire modelli che non solo si adattano bene ai dati esistenti ma sono anche robusti contro l'overfitting. Poiché la vera distribuzione $ p(y) $ è sconosciuta, ricorriamo a metodi di approssimazione come la validazione incrociata per stimare questa metrica. Questi metodi simuleranno l'effetto di testare il modello su nuovi dati, fornendo un'indicazione affidabile della sua capacità predittiva.

#### Metodi di Approssimazione per la Stima dell'Expected Log Predictive Density (ELPD)

Per valutare accuratamente la capacità predittiva di un modello statistico, è essenziale utilizzare metodi di approssimazione che stimino l'Expected Log Predictive Density (ELPD). Tra i metodi più comuni troviamo il **Leave-One-Out Cross-Validation (LOO-CV)**, **Akaike Information Criterion (AIC)**, e **Watanabe-Akaike Information Criterion (WAIC)**. Ognuno di questi metodi offre un approccio diverso per bilanciare la complessità del modello con la sua capacità di adattamento e predizione.

### Leave-One-Out Cross-Validation (LOO-CV)

Il LOO-CV è particolarmente utile in scenari dove il set di dati non è molto ampio. Questo metodo prevede di rimuovere un dato alla volta dal set di addestramento, utilizzare il modello per predire il valore di questo dato escluso, e valutare la bontà di questa predizione. L'approccio massimizza l'utilizzo dei dati disponibili, rendendolo ideale per modelli complessi e campioni di dimensioni ridotte. Attraverso il LOO-CV, si può ottenere una stima dell'ELPD che riflette quanto bene il modello possa comportarsi con nuovi dati non ancora osservati, integrando la log-verosimiglianza predittiva:

$$
\text{ELPD} = \sum_{i=1}^N \log p(y_i \mid y_{-i}),
$$

dove $ y_i $ è il dato escluso e $ y_{-i} $ rappresenta tutti gli altri dati nel set di addestramento.

### Akaike Information Criterion (AIC) e Watanabe-Akaike Information Criterion (WAIC)

Sia AIC che WAIC sono criteri informativi che valutano la qualità di un modello basandosi sui dati osservati. Entrambi i criteri cercano di bilanciare la bontà di adattamento del modello con la sua complessità. L'AIC, in particolare, penalizza i modelli in base al numero dei parametri, fornendo una misura che favorisce modelli meno complessi ma altrettanto capaci di adattarsi ai dati:

$$
\text{AIC} = 2k - 2\ln(L),
$$

dove $ k $ è il numero di parametri del modello e $ L $ è la massima verosimiglianza raggiunta dal modello.

Il WAIC estende questo concetto considerando una forma di media pesata della devianza logaritmica, più robusta nei contesti di dati gerarchici o modelli bayesiani:

$$
\text{WAIC} = -2 \left( \text{LPD} - \text{penalità di variazione} \right),
$$

dove LPD è la log-probabilità predittiva del dato e la penalità di variazione quantifica l'effetto della varianza dei parametri sulle predizioni del modello.

### Valutazione Comparativa e Applicazioni Pratiche

Questi metodi forniscono diverse prospettive sulla stima dell'ELPD, ognuno con i suoi vantaggi a seconda del contesto e della complessità del modello considerato. Il LOO-CV è particolarmente prezioso per modelli complessi o set di dati limitati, mentre AIC e WAIC offrono approcci più rapidi e meno computazionalmente intensivi, adatti per valutazioni preliminari o quando si dispone di grandi set di dati.

In sintesi, la scelta del metodo di approssimazione dipende dall'obiettivo specifico della valutazione del modello, dalla natura dei dati disponibili e dalle risorse computazionali a disposizione. Attraverso l'uso di questi metodi, possiamo ottenere una stima più accurata e affidabile della capacità di un modello di predire nuovi dati, essenziale per la validazione e la selezione del modello in contesti scientifici e applicativi.

## Considerazioni Conclusive

In conclusione, la divergenza di Kullback-Leibler (KL) fornisce una valutazione focalizzata sull'adattamento di un modello ai dati osservati, mentre l'Expected Log Predictive Density (ELPD) e i relativi metodi di approssimazione, quali LOO-CV (Leave-One-Out Cross-Validation), AIC (Akaike Information Criterion) e WAIC (Watanabe-Akaike Information Criterion), si concentrano sulla valutazione della capacità di generalizzazione del modello a dati futuri non ancora osservati. L'impiego combinato di queste metriche consente di ottenere una valutazione olistica della performance di un modello, abbracciando sia la sua precisione nell'adattarsi ai dati attuali sia la sua efficacia predittiva su nuovi dati. 

## Informazioni sull'Ambiente di Sviluppo

In [17]:
%load_ext watermark
%watermark -n -u -v -iv -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Wed Apr 24 2024

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

numpy      : 1.26.4
pandas     : 2.2.2
statsmodels: 0.14.1
matplotlib : 3.8.4
pymc       : 5.13.0
arviz      : 0.18.0
seaborn    : 0.13.2
scipy      : 1.13.0

Watermark: 2.4.3

